In [1]:
# move this down
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv('scotch_review.csv')

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pickle

In [3]:
URL0 = 'https://flaviar.com/bottles/scotch'

page0 = requests.get(URL0)

soup0 = BeautifulSoup(page0.text, 'lxml')

In [4]:
# scraps website for bottle edition

bottle_names = []
bottle_editions = []
bottle_types = []



class0 = soup0.find('div', {'id': 'content', 'class': 'overflow-hidden'})
class1 = class0.find('div', {'class': 'fullWidth pattern-bg sand-bg-mobile padding-top-50 padding-top-mobile-40 padding-bottom-50'})
class2 = class1.find('div', {'class': 'container'})
class3 = class2.find('div', {'class': 'row'})
class4 = class3.find('div', {'class': 'col-md-12'})
class5 = class4.find('div', {'id': 'previousboxes', 'class': 'row margin-left-0 margin-right-0 padding-left-mobile-10 padding-right-mobile-10 bottles'})
class6 = class5.find('div', {'class': 'row'})
class7 = class6.find_all('div', {'class': 'col-lg-3 col-md-4 col-sm-6'})


for i in class7:
    class8 = i.find('a', {'class': 'single-item bottle'})    
    class9 = class8.find('div', {'class': 'holder'})
    class10 = class9.find('div', {'class': 'about flex flex--space-between'})
    class11 = class10.find('div', {'class': 'desc'})
    bottle_name = class11.find('div', {'class': 'name'})
    bottle_edition = class11.find('div', {'class': 'edition'})
    bottle_type = class11.find('div', {'class': 'type'})
    
    bottle_names.append(bottle_name.text)
    bottle_editions.append(bottle_edition.text)
    bottle_types.append(bottle_type.text)
    



In [5]:
# Creates url for all editions in selected bottle type(s)
# Type choosen when URL0 is defined

url_editions = []
BOTTLE_COUNT = len(bottle_editions)
for n in range(BOTTLE_COUNT):
    url_editions.append((str('https://flaviar.com/' + bottle_names[n] + '/' + bottle_editions[n]).replace(' ','-')))
    

In [6]:
print(url_editions[0])

https://flaviar.com/The-Naked-Grouse/The-Naked-Grouse


In [7]:
appearances = []
noses = []
palates = []
finishes = []





for url in range(len(url_editions)):
    
    
    URL1 = url_editions[url]
    
    page1 = requests.get(URL1)

    soup1 = BeautifulSoup(page1.text, 'lxml')
    
    
    profile_class0 = soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'})
    profile_class1 = profile_class0.find('div', {'id': 'info', 'class': 'container'})
    profile_class2 = profile_class1.find('div', {'class': 'row shrink flex flex--stretch margin-bottom-50 margin-bottom-mobile-0'})
    profile_class3 = profile_class2.find('div', {'class': 'col-md-8 col-sm-12 col-xs-12 element info-description flex _flex-2 fit2EdgeOnMobile'})
    profile_class4 = profile_class3.find('div', {'class': 'introduce pattern-bg equal moduleTabs collapsable-content-box'})
    profile_class5 = profile_class4.find('id': 'tasting-notes','div', {'class': 'body collapse in more-body-padding'})
    profile_class6 = profile_class5.find('div', {'class': 'p-content'})
    
    
    
    
profiles = [p for p in profile_class6]


print('As list:\n', profiles)
print()


SyntaxError: invalid syntax (<ipython-input-7-804cdea98d99>, line 25)

In [8]:
#print(soup1)
print(url_editions[0])
#print(soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'}))

https://flaviar.com/The-Naked-Grouse/The-Naked-Grouse


In [9]:
URL1 = url_editions[0]

page1 = requests.get(URL1)

soup1 = BeautifulSoup(page1.text, 'lxml')

In [10]:
profile_class0 = soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'})
profile_class1 = profile_class0.find('div', {'id': 'info', 'class': 'container'})
profile_class2 = profile_class1.find('div', {'class': 'row shrink flex flex--stretch margin-bottom-50 margin-bottom-mobile-0'})
profile_class3 = profile_class2.find('div', {'class': 'col-md-8 col-sm-12 col-xs-12 element info-description flex _flex-2 fit2EdgeOnMobile'})
profile_class4 = profile_class3.find('div', {'class': 'introduce pattern-bg equal moduleTabs collapsable-content-box'})
profile_class5 = profile_class4.find('div', {'id': 'tasting-notes', 'class': 'body collapse in more-body-padding'})
profile_class6 = profile_class5.find('div', {'class': 'p-content'})
#profile_class7 = profile_class6.find('div', {'strong': 'Appearance / Color'})

#appearances.append(profile_class7.

#dir()



#print(appearances)

print('As object:\n'+ str(profile_class6))

As object:
<div class="p-content">
<div>
<strong>Appearance / Color</strong></div>
<div>
	Amber</div>
<div>
	 </div>
<div>
<strong>Nose / Aroma / Smell</strong></div>
<div>
	Smooth sherry aroma followed by notes of buttered toast, malt, peach and custard.</div>
<div>
	 </div>
<div>
<strong>Flavor / Taste / Palate</strong></div>
<div>
	Sherried palate with notes of toffee, sultanas, raspberry jam, custard and caramel.</div>
<div>
	 </div>
<div>
<strong>Finish</strong></div>
<div>
	Medium finish with noes of oak and cocoa.</div>
</div>


In [102]:
appearances = []
noses = []
palates = []
finishes = []


for p in profile_class6.find_all('div'):
    
    if ( p.text == '\nAppearance / Color'):
        print('FOUND APPEARANCE\n')
        appearances.append(str(profile_class6.div.next_sibling.next_sibling.text).strip('\r\n\t'))
        
    elif (p.text == '\nNose / Aroma / Smell'):
        print('FOUND NOSE\n')
        noses.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                         next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                         text).strip('\r\n\t'))
        
    elif (p.text == '\nFlavor / Taste / Palate'):
        print('FOUND FLAVOR\n')
        palates.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                           next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                           next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                           next_sibling.text).strip('\r\n\t')) 
        
    elif (p.text == '\nFinish'):
        print('FOUND FINISH')
        finishes.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                            next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                            next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                            next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                            next_sibling.next_sibling.text).strip('\r\n\t'))
        
        continue

print(appearances)
print(noses)
print(palates)
print(finishes)


FOUND APPEARANCE

FOUND NOSE

FOUND FLAVOR

FOUND FINISH
['Amber']
['Smooth sherry aroma followed by notes of buttered toast, malt, peach and custard.']
['Sherried palate with notes of toffee, sultanas, raspberry jam, custard and caramel.']
['Medium finish with noes of oak and cocoa.']


In [77]:
print(appearances)
print(noses)
print(palates)
print(finishes)
dir(str)

['Amber']
[]
[]
[]


['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',


In [26]:
pickle.dump(profile_class6.text, open( "tasting-notes.p", "wb" ))
#print(pickle.load(open( "tasting-notes.p", "rb" )))



pickle.dump(bottle_editions, open( "bottle-editions.p", "wb" ))
#print(pickle.load(open( "bottle-editions.p", "rb" )))

In [109]:
dir(profile_class6)

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 '